In [1]:
from keras.applications.inception_v3 import InceptionV3
model = InceptionV3()
print(model.summary())

C:\Users\Zhouxianwei\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [2]:
import numpy as np
import time
from keras.applications.inception_v3 import InceptionV3 #CHANGE
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.layers import Dense,Activation,Flatten
from keras.layers import merge,Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import resample #CHANGE
from sklearn.cross_validation import train_test_split
import os

data_path=r"D:\semester3\capstone\FGVC_data"
data = os.listdir(data_path)

img_data_list=[]

for dataset in data:
    img_list = os.listdir(data_path+'/'+dataset)
    print('loaded images'+'{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path+'/'+dataset+'/'+img
        img = image.load_img(img_path,target_size=(299,299))
        x = image.img_to_array(img)
        x = np.expand_dims(x,axis=0)
        x = preprocess_input(x)
        #print('Input image shape',x.shape)
        img_data_list.append(x)
        
img_data = np.array(img_data_list)
print(img_data.shape)
img_data = np.rollaxis(img_data,1,0)
print(img_data.shape)
img_data = img_data[0]
print(img_data.shape)

num_classes = 10
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')


labels[0:17] = 0
labels[17:783] =1
labels[783:893] =2
labels[893:915]=3
labels[915:937]=4
labels[937:1050]=5
labels[1050:1079]=6
labels[1079:1094]=7
labels[1094:1120]=8
labels[1120:1489]=9

names=['2746','2747','2748','2749','2750','2751','2752','2753','2754','2755']



C:\Users\Zhouxianwei\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


loaded images2746

loaded images2747

loaded images2748

loaded images2749

loaded images2750

loaded images2751

loaded images2752

loaded images2753

loaded images2754

loaded images2755

(1489, 1, 299, 299, 3)
(1, 1489, 299, 299, 3)
(1489, 299, 299, 3)


In [3]:
Y = np_utils.to_categorical(labels,num_classes)

x,y = resample(img_data,Y,random_state=2,replace=False)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

image_input = Input(shape=(299,299,3))
model=InceptionV3(input_tensor=image_input,weights='imagenet')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 149, 149, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [4]:
last_layer = model.get_layer('avg_pool').output
out =Dense(num_classes,activation='softmax',name='output')(last_layer)
custom_InceptionV3_model = Model(image_input,out)
custom_InceptionV3_model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 149, 149, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

__________________________________________________________________________________________________
batch_normalization_155 (BatchN (None, 17, 17, 192)  576         conv2d_155[0][0]                 
__________________________________________________________________________________________________
batch_normalization_158 (BatchN (None, 17, 17, 192)  576         conv2d_158[0][0]                 
__________________________________________________________________________________________________
batch_normalization_163 (BatchN (None, 17, 17, 192)  576         conv2d_163[0][0]                 
__________________________________________________________________________________________________
batch_normalization_164 (BatchN (None, 17, 17, 192)  576         conv2d_164[0][0]                 
__________________________________________________________________________________________________
activation_155 (Activation)     (None, 17, 17, 192)  0           batch_normalization_155[0][0]    
__________

In [5]:
for layer in custom_InceptionV3_model.layers[:-1]:
    layer.trainable = False
    
print(custom_InceptionV3_model.layers[3].trainable)

custom_InceptionV3_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

t=time.time()
hist=custom_InceptionV3_model.fit(X_train,y_train,batch_size=32,epochs=5,verbose=1,validation_data=(X_test,y_test))
print('Taining time:%s' %(t-time.time()))
(loss,accuracy)=custom_InceptionV3_model.evaluate(X_test,y_test,batch_size=10,verbose=1)
print('loss={:.4f},accuracy:{:.4f}%'.format(loss,accuracy*100))

False
Train on 1191 samples, validate on 298 samples
Epoch 1/5
1191/1191 [==============================] - 1153s 968ms/step - loss: 1.3490 - acc: 0.5592 - val_loss: 1.4036 - val_acc: 0.5336
Epoch 2/5
1191/1191 [==============================] - 1157s 971ms/step - loss: 1.1343 - acc: 0.6289 - val_loss: 1.2546 - val_acc: 0.6309
Epoch 3/5
1191/1191 [==============================] - 1232s 1s/step - loss: 1.0076 - acc: 0.6616 - val_loss: 1.3168 - val_acc: 0.6040
Epoch 4/5
1191/1191 [==============================] - 1271s 1s/step - loss: 0.9239 - acc: 0.6767 - val_loss: 1.2124 - val_acc: 0.5940
Epoch 5/5
1191/1191 [==============================] - 1362s 1s/step - loss: 0.8905 - acc: 0.6919 - val_loss: 1.1179 - val_acc: 0.6275
Taining time:-6176.265336275101
298/298 [==============================] - 259s 869ms/step
loss=1.1179,accuracy:62.7517%
